In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import xgboost as xgb
from xgboost import XGBClassifier, XGBRFClassifier
import lightgbm as lgb

%matplotlib inline 

In [32]:
df = pd.read_csv('CompleteDataset.csv')
df.head()

C:\Users\abc\AppData\Local\Temp\ipykernel_7492\104370077.py:1: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CompleteDataset.csv')


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df.columns

Index(['Unnamed: 0', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'GK diving', 'GK handling', 'GK kicking',
       'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM', 'ID',
       'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB',
       'Preferred Positions', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM',
       'RS', 'RW', 'RWB', 'ST'],
      dtype='object')

In [34]:
# GK attributes are not our interest
columns_needed = ['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys', 'Preferred Positions']

# attack attribute first, then defence, then mixed
columns_needed_rearranged = ['Aggression','Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'Heading accuracy', 'Long shots','Penalties', 'Shot power', 'Volleys', 
       'Short passing', 'Long passing',
       'Interceptions', 'Marking', 'Sliding tackle', 'Standing tackle',
       'Strength', 'Vision', 'Acceleration', 'Agility', 
       'Reactions', 'Stamina', 'Balance', 'Ball control','Composure','Jumping', 
       'Sprint speed', 'Positioning','Preferred Positions']

df = df[columns_needed_rearranged]
df.head()

,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,63,85,81,91,94,76,88,92,85,94,...,89,96,92,63,93,95,95,91,95,ST LW
1,48,77,89,97,95,90,71,88,74,85,...,90,95,73,95,95,96,68,87,93,RW
2,56,75,81,96,89,84,62,77,81,80,...,96,88,78,82,95,92,61,90,90,LW
3,78,77,86,86,94,84,77,86,85,87,...,86,93,89,60,91,83,69,77,92,ST
4,29,15,14,30,13,11,25,16,47,25,...,52,85,44,35,48,70,78,61,12,GK


In [35]:
#we dont classify GK data
df['Preferred Positions'] = df['Preferred Positions'].str.strip()
df = df[df['Preferred Positions'] != 'GK']
df.head()

,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,63,85,81,91,94,76,88,92,85,94,...,89,96,92,63,93,95,95,91,95,ST LW
1,48,77,89,97,95,90,71,88,74,85,...,90,95,73,95,95,96,68,87,93,RW
2,56,75,81,96,89,84,62,77,81,80,...,96,88,78,82,95,92,61,90,90,LW
3,78,77,86,86,94,84,77,86,85,87,...,86,93,89,60,91,83,69,77,92,ST
5,80,62,77,85,91,84,85,83,81,88,...,78,91,79,80,89,87,84,83,91,ST


In [36]:
#check missing data
df.isnull().values.any()

False

In [37]:
p = df['Preferred Positions'].str.split().apply(lambda x: x[0]).unique()
p

array(['ST', 'RW', 'LW', 'CDM', 'CB', 'RM', 'CM', 'LM', 'LB', 'CAM', 'RB',
       'CF', 'RWB', 'LWB'], dtype=object)

In [38]:
# copy a structure
df_new = df.copy()
df_new.drop(df_new.index, inplace=True)

for i in p:
    df_temp = df[df['Preferred Positions'].str.contains(i)]
    df_temp['Preferred Positions'] = i
    df_new = df_new.append(df_temp, ignore_index=True)
    
df_new.iloc[::500, :]
            



C:\Users\abc\AppData\Local\Temp\ipykernel_7492\3863530977.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Preferred Positions'] = i
C:\Users\abc\AppData\Local\Temp\ipykernel_7492\3863530977.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(df_temp, ignore_index=True)


,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,63,85,81,91,94,76,88,92,85,94,...,89,96,92,63,93,95,95,91,95,ST
500,69,44,55,66,80-3,54,65,75,75,82,...,68-2,67,86-3,64+4,68,63,69,75-3,70+3,ST
1000,70,62,59,67,68,43,71,63,67,66,...,68,68,74,70,69,67,72,63,75,ST
1500,46,55,41,69,67,41,59,62,61,64,...,67,68,47,71,71,64,73,67,64,ST
2000,32,59,47,67,53,42,40,42,51,50,...,81,58,66,79,65,44,60,84,61,ST
2500,64,62,48,66,61,31,55,52,49,65,...,70,45,74,66,60,48,68,69,58,ST
3000,42,31,36,48,61,27,59,49,53,53,...,58,53,55,56,52,49,78,56,49,ST
3500,66,65,66,75,66,55,59,69,53,79,...,73,68,79,62,74,65,75,91,68,RW
4000,41,59,38,61,57,36,58,52,57,59,...,70,65,72,69,58,49,54,77,60,RW
4500,76,73,57,72,42,59,60,41,35,57,...,82,65,71,83,72,64,70,79,57,LW


In [39]:
# Some of the attributes have '+/-' sign, let's perform the calculation rather than keeping them as string:

cols = [col for col in df_new.columns if col not in ['Preferred Positions']]

for i in cols:
    df_new[i] = df_new[i].apply(lambda x: eval(x) if isinstance(x,str) else x)

df_new.iloc[::500, :]

,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,63,85,81,91,94,76,88,92,85,94,...,89,96,92,63,93,95,95,91,95,ST
500,69,44,55,66,77,54,65,75,75,82,...,66,67,83,68,68,63,69,72,73,ST
1000,70,62,59,67,68,43,71,63,67,66,...,68,68,74,70,69,67,72,63,75,ST
1500,46,55,41,69,67,41,59,62,61,64,...,67,68,47,71,71,64,73,67,64,ST
2000,32,59,47,67,53,42,40,42,51,50,...,81,58,66,79,65,44,60,84,61,ST
2500,64,62,48,66,61,31,55,52,49,65,...,70,45,74,66,60,48,68,69,58,ST
3000,42,31,36,48,61,27,59,49,53,53,...,58,53,55,56,52,49,78,56,49,ST
3500,66,65,66,75,66,55,59,69,53,79,...,73,68,79,62,74,65,75,91,68,RW
4000,41,59,38,61,57,36,58,52,57,59,...,70,65,72,69,58,49,54,77,60,RW
4500,76,73,57,72,42,59,60,41,35,57,...,82,65,71,83,72,64,70,79,57,LW


In [40]:
df_new_normalized = df_new.iloc[:,:-1].div(df_new.iloc[:,:-1].sum(axis=1), axis=0)
mapping = {'ST': 1, 'RW': 1, 'LW': 1, 'RM': 1, 'CM': 1, 'LM': 1, 'CAM': 1, 'CF': 1, 'CDM': 0, 'CB': 0, 'LB': 0, 'RB': 0, 'RWB': 0, 'LWB': 0}
df_new_normalized['Preferred Positions'] = df_new['Preferred Positions']
df_new_normalized = df_new_normalized.replace({'Preferred Positions': mapping})

df_new_normalized.iloc[::1000,]


,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,0.027815,0.037528,0.035762,0.040177,0.041501,0.033554,0.038852,0.040618,0.037528,0.041501,...,0.039294,0.042384,0.040618,0.027815,0.041060,0.041943,0.041943,0.040177,0.041943,1
1000,0.039282,0.034792,0.033109,0.037598,0.038159,0.024130,0.039843,0.035354,0.037598,0.037037,...,0.038159,0.038159,0.041526,0.039282,0.038721,0.037598,0.040404,0.035354,0.042088,1
2000,0.021291,0.039255,0.031271,0.044578,0.035263,0.027944,0.026613,0.027944,0.033932,0.033267,...,0.053892,0.038589,0.043912,0.052562,0.043247,0.029275,0.039920,0.055888,0.040585,1
3000,0.031065,0.022929,0.026627,0.035503,0.045118,0.019970,0.043639,0.036243,0.039201,0.039201,...,0.042899,0.039201,0.040680,0.041420,0.038462,0.036243,0.057692,0.041420,0.036243,1
4000,0.027009,0.038867,0.025033,0.040184,0.037549,0.023715,0.038208,0.034256,0.037549,0.038867,...,0.046113,0.042819,0.047431,0.045455,0.038208,0.032279,0.035573,0.050725,0.039526,1
5000,0.022193,0.030679,0.037859,0.039164,0.030026,0.035901,0.035901,0.035248,0.041123,0.045692,...,0.048303,0.037859,0.050261,0.040470,0.038512,0.037859,0.033290,0.051567,0.046997,1
6000,0.036757,0.031351,0.030811,0.035676,0.029189,0.031351,0.025946,0.034595,0.037297,0.036757,...,0.036757,0.038919,0.037297,0.039459,0.037838,0.041622,0.039459,0.022703,0.032432,0
7000,0.035219,0.038106,0.032333,0.036374,0.034642,0.020208,0.034065,0.031755,0.037529,0.031755,...,0.035219,0.034642,0.037529,0.033487,0.038684,0.032910,0.038106,0.038106,0.038106,0
8000,0.039876,0.027447,0.024858,0.033143,0.033661,0.023822,0.035733,0.033143,0.025893,0.039876,...,0.026929,0.037286,0.041947,0.029518,0.039358,0.034697,0.038322,0.032108,0.030554,0
9000,0.046556,0.036352,0.017857,0.036990,0.014031,0.026786,0.043367,0.028061,0.039541,0.039541,...,0.024872,0.043367,0.024235,0.027423,0.037628,0.038903,0.036352,0.021046,0.028061,0


In [41]:
X_train, X_test, y_train, y_test = train_test_split(df_new_normalized.iloc[:,:-1], df_new_normalized.iloc[:,-1], random_state=0)

print('X train shape: {}'.format(X_train.shape))
print('X test shape: {}'.format(X_test.shape))
print('y train shape: {}'.format(y_train.shape))
print('y test shape: {}'.format(y_test.shape))


X train shape: (20438, 29)
X test shape: (6813, 29)
y train shape: (20438,)
y test shape: (6813,)


In [42]:
# Train a decision tree model
dtc = DecisionTreeClassifier(max_depth=10, random_state=42)
dtc.fit(X_train, y_train)
dtc_y_pred = dtc.predict(X_test)
dtc_accuracy = accuracy_score(y_test, dtc_y_pred)
print("Decision Tree Accuracy:", dtc_accuracy)


Decision Tree Accuracy: 0.8232790253926318


In [43]:
# Train a random forest model
rfc = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42)
rfc.fit(X_train, y_train)
rfc_y_pred = rfc.predict(X_test)
rfc_accuracy = accuracy_score(y_test, rfc_y_pred)
print("Random Forest Accuracy:", rfc_accuracy)


Random Forest Accuracy: 0.8357551739321885


In [44]:
# Train a LightGBM decision tree model
lgb_dt = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, random_state=42)
lgb_dt.fit(X_train, y_train)
lgb_dt_y_pred = lgb_dt.predict(X_test)
lgb_dt_accuracy = accuracy_score(y_test, lgb_dt_y_pred)
print("LightGBM Decision Tree Accuracy:", lgb_dt_accuracy)

LightGBM Decision Tree Accuracy: 0.8376632907676501


In [45]:
# Train a LightGBM random forest model
lgb_rf = lgb.LGBMClassifier(boosting_type='rf', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, random_state=42, bagging_fraction=0.8, bagging_freq=10)
lgb_rf.fit(X_train, y_train)
lgb_rf_y_pred = lgb_rf.predict(X_test)
lgb_rf_accuracy = accuracy_score(y_test, lgb_rf_y_pred)
print("LightGBM Random Forest Accuracy:", lgb_rf_accuracy)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
LightGBM Random Forest Accuracy: 0.8563041244679289
